Dr Oliviero Andreussi, olivieroandreuss@boisestate.edu

Boise State University, Department of Chemistry and Biochemistry

# Fitting and Data Analysis for the UV-Vis Particle in a Box Experiment {-}

Before we start, let us import the main modules that we will need for this lecture. You may see some new modules in the list below, we will add more details in the right sections.

In [ ]:
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

We need to load the Google Drive and access a file that contains the information on the molecules studies and the measured absorption wavelengths. You can use the same set that I am using by downloading it from Canvas, [here](https://boisestatecanvas.instructure.com/courses/20544/files/10392339?wrap=1). Or you can use your own files. I am assuming the file in question will be located in a `ParticleBox_Data/` subfolder in your `Colab Notebook/` folder. I will be using a file named `particleinabox.csv`, make sure to change this name if you use a different file. Also, in the file you will need to change the absorption wavelengths into your own measured results, as they are all set to 500 nm in the template file.

In [ ]:
# Load the google drive with your files 
from google.colab import drive
drive.mount('/content/drive')
# The following needs to be the path of the folder with all your datafile in .csv format
path='/content/drive/MyDrive/Colab Notebooks/ParticleBox_Data/'
file='particleinabox.csv'

The following module needs to be installed on Colab. We won't need it too much for this analysis, but they offer a lot of nice features for chemistry programming. 

In [ ]:
# These modules are needed to draw chemical formulas
#!pip install rdkit cirpy
from rdkit import Chem
from rdkit.Chem import Draw
import cirpy

In particular we can use them to draw the molecules in our experiments

In [ ]:
data=pd.read_csv(path+file)
data

If we want to visualize the different molecules used in the experiment, we can convert the CAS codes into smiles and use them to generate the chemical structures

In [ ]:
data['smiles']=[ cirpy.resolve( x['cas'],'smiles') for i,x in data.iterrows() ]
# unfortunately one of the molecules is not in the databases and we need to enter the smiles by hand
data.loc[5,'smiles']='[I-].CCN1C=CC(=CC=CC=CC2=CC=[N+](C3=CC=CC=C23)CC)C4=CC=CC=C41'
data

We can now visualize the molecule that we like

In [ ]:
index=0
display(Draw.MolToImage( Chem.MolFromSmiles(data.loc[index,'smiles']), size=(300, 300) ))

In [ ]:
def lambda_FE(p,alpha=0.):
    """
    Function to compute the absorption wavelenght (in nm)
    as a function of the number of carbon atoms in the conjugated chain
    and an optional parameter alpha that accounts for some wiggle room
    due to the aromatic rings
    """
    return 63.7*(p+3+alpha)**2/(p+4)

Given the function above, we can compute a new column of the dataframe with the predicted lambdas in one command

In [ ]:
data['lambda_FE']=lambda_FE(data['p'])
data

We can visualize the results as usual

In [ ]:
plt.scatter(data['p'],data['lambda'])
plt.plot(data['p'],data['lambda_FE'],color='red')
plt.show()

Given our experimental and predicted values of lambda, we can compute the sum of squares as follows:

In [ ]:
TSS=np.sum((data['lambda']-data['lambda'].mean())**2)
print("The total sum of squares is {:10.4f} ".format(TSS))

In [ ]:
RSS=np.sum((data['lambda']-data['lambda_FE'])**2)
print("The residual sum of squares is {:10.4f} ".format(RSS))

In [ ]:
R2=1-(RSS/TSS)
print("The coefficient of determination R2 for the particle in a box model is {:5.4f} ".format(R2))

## Huckel Model for Conjugated Molecules {-}

While for the Full Report you will only worry about modeling your experiments with a Free Electron (Particle in a Box) kind of model, here we will try to understand how we can use a slightly more advanced quantum mechanical model to predict absorption wavelengths. The Huckel model is a semi-empirical quantum mechanical model that has been developed to describe conjugted systems. While the model relies on some adjustable parameters and involves several possibly strong assumptions, it allows to reproduce some key properties of conjugated and aromatic molecules.

You can learn more on the theory and assumptions of the model in the lectures and in the following online resources: 
* MIT Physical Chemistry on [The Huckel Molecular Orbital Theory](https://dspace.mit.edu/bitstream/handle/1721.1/120336/5-61-fall-2013/contents/lecture-notes/MIT5_61F13_Lecture27-28.pdf)
* P-Chem Lab from Duke University [The Huckel Approximation](https://chem.libretexts.org/Courses/Duke_University/CHEM310L_-_Physical_Chemistry_I_Lab_Manual/04%3A_Absorption_Spectrum_of_Conjugated_Dyes/4.07%3A_Appendix_B_-_The_Huckel_Approximation)
* Columbia Notes on [The Huckel Approximation](http://www.columbia.edu/itc/chemistry/chem-c2407_archive/recitations/huckel.pdf)


The key component of the Huckel model is the fact that the molecular orbitals are univoquely determined by the topology of the conjugated network. The model assumes that we only consider the $p_z$ orbitals of each conjugated atom and we build molecular orbitals from them. As a further approximation, only atomic orbitals that are in connected atoms are allowed to 'interact'. Eventually, we assume that all atoms and bonds are equivalent, so that the relative components of the molecular Hamiltonian are identical. The Hamiltonian of the system can thus be represented as a matrix, where diagonal cells $H_{ii}$ have identical values, while off-diagonal elements $H_{ij}$ are different from zero only if there is a bond between atom $i$ and atom $j$. For carbon based molecules, the model only relies on two parameters $\alpha$ and $\beta$. However, the difference in energy between the electronic states only depends on the latter parameter, which is usually estimated to be $\beta\approx-70.4\; kcal/mol$.

For the case of butane we can build the Huckel Hamiltonian as follows:

In [ ]:
n_conjugated = 4
alpha = -2
beta = -1
topology = np.zeros((n_conjugated,n_conjugated))
diagonal = np.ones(n_conjugated)*alpha
offdiagonal = np.ones(n_conjugated-1)*beta
topology = topology + np.diag(diagonal,0) + np.diag(offdiagonal,1) + np.diag(offdiagonal,-1)
print(topology)

Molecular orbitals are given by the linear combinations of atomic orbitals that make the Hamiltonian diagonal and minimize the energy of the system. Diagonalization of the Hamiltonian is a linear algebra problem (eigenvalue problem). We can use numpy to compute eigenvalues (a.k.a. orbital energies) and eigenvectors (a.k.a. the values of the coefficients that enter the definition of the molecular orbitals).

In [ ]:
eigenvalues,eigenvectors=np.linalg.eig(topology)
print(eigenvalues)

You can note that the energies above are all different from the starting energy of the $p_z$ orbitals, which was arbitrary set to $-200\;kcal/mol$. For this kind of Hamiltonians, the eigenvalues can be expressed with an analytical formula $E_j=\alpha+2\beta\cos\left(\frac{\pi}{N+1}J\right)$, with $J=1,2,\dots ,N$. For longer conjugated chains, assuming that we have one electron for each atomic orbital (so number of electrons is $N$ and only the first $N/2$ orbitals are filled), we can estimate the energy difference associated with the absorption maximum as $\Delta E=-4\beta\sin\left(\frac{\pi}{2}\frac{1}{N+1}\right)$. When accounting for the different units, this energy corresponds to a wavelenght of $\lambda^{HMO}=-\frac{28585\;(kcal/mol)}{2\pi\beta\;(kcal/mol)}N$. As for the free electron model, the Huckel Molecular Orbital approach predicts an absorption wavelength that is linear in the number of conjugated centers. The tunable parameter in this case is the $\beta$ constant. 

However, the Huckel model can be automatically extended to account for the aromatic rings and it can, with additional parameters, be extended to include heteroatoms. For example, it is worth looking at the results of the Huckel model for a benzene molecule.

In [ ]:
n_conjugated = 6
alpha = -2.
beta = -1.
topology = np.zeros((n_conjugated,n_conjugated))
diagonal = np.ones(n_conjugated)*alpha
offdiagonal = np.ones(n_conjugated-1)*beta
topology[0,n_conjugated-1]=beta
topology[n_conjugated-1,0]=beta
topology = topology + np.diag(diagonal,0) + np.diag(offdiagonal,1) + np.diag(offdiagonal,-1)
print(topology)

Note that in the following we re-order the results of the eigenvalue problem so that we can see the states in order of their energies (from the largest less stable to the smallest more stable)

In [ ]:
eigenvalues,eigenvectors=np.linalg.eig(topology)
idx = eigenvalues.argsort()[::-1]   
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:,idx]

We can use matplotlib to visualize the coefficients in terms of their signs and magnitudes as follows.

In [ ]:
from matplotlib.colors import BoundaryNorm
bounds = np.arange(np.min(eigenvectors),np.max(eigenvectors),.05)
cmap = plt.get_cmap('seismic')
idx=np.searchsorted(bounds,0)
bounds=np.insert(bounds,idx,0)
norm = BoundaryNorm(bounds, cmap.N)
plt.imshow(eigenvectors,interpolation='none',norm=norm,cmap=cmap)
plt.colorbar()
plt.show()


If you have time, you could try to build a topology matrix for a few of the molecules involved in the experiments and compare the HOMO-LUMO energy difference with the formula reported above for a linear conjugated chain.